In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt

In [ ]:
path = "input/"
data = pd.read_csv(path+"sudoku.csv")
try:
    data = pd.DataFrame({"quizzes":data["puzzle"],"solutions":data["solution"]})
except:
    pass

data.info()

In [ ]:
data.head()

In [ ]:
print("Quiz:\n",np.array(list(map(int,list(data['quizzes'][0])))).reshape(9,9))
print("Solution:\n",np.array(list(map(int,list(data['solutions'][0])))).reshape(9,9))


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info
        
        self.data_path = path
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train': 
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [ ]:
train_idx = int(len(data)*0.95)
data = data.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(data.iloc[:train_idx], subset = "train", batch_size=32)
validation_generator = DataGenerator(data.iloc[train_idx:], subset = "train",  batch_size=32)        

In [ ]:
type(training_generator)

In [ ]:
training_generator.__getitem__(4)[0].shape

In [2]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = tf.keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 32)          320       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 32)          128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0         
_________________________________________________________________
dense (Dense)                (None, 729)               7

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath2 = "best_weights.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [checkpoint1,checkpoint2,reduce_lr]

In [ ]:
history = model.fit_generator(training_generator, validation_data = validation_generator, epochs = 3, verbose=1,callbacks=callbacks_list )

In [3]:
model.load_weights('best_weights.hdf5')

In [10]:
def norm(a):
    return (a/9)-.5

def denorm(a):
    return (a+.5)*9

def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = sample

    cp =0
    while(cp < 1):
    
        out = model.predict(feat.reshape((1,9,9,1))) 
        
        out = out.squeeze()
        
        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
        ind = np.argmax(prob_new)
        print(ind)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        print(x+1, y+1 , val)
        feat[x][y] = val
        print(feat)
        feat = norm(feat)
        
        cp+=1
    return pred

def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

def solve_sudoku(game):
    try:
        game = game.replace('\n', '')
        game = game.replace(' ', '')
        game = np.array([int(j) for j in game]).reshape((9,9,1))   
    except: 
        pass
   
    game = game.reshape((9,9,1))
   
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [ ]:
game = '''
          0 0 0 7 0 0 0 9 6
          0 0 3 0 6 9 1 7 8
          0 0 7 2 0 0 5 0 0
          0 7 5 0 0 0 0 0 0
          9 0 1 0 0 0 3 0 0
          0 0 0 0 0 0 0 0 0
          0 0 9 0 0 0 0 0 1
          3 1 8 0 2 0 4 0 7
          2 4 0 0 0 5 0 0 0
      '''

In [ ]:

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

In [12]:
entree_facile = np.array([[9, 0, 6, 4, 0, 5, 0, 8, 0],
                 [0, 7, 2, 0, 0, 0, 0, 4, 1],
                 [3, 4, 0, 0, 0, 2, 0, 0, 0],
                 [0, 0, 4, 1, 6, 0, 3, 0, 0],
                 [7, 0, 1, 0, 4, 0, 9, 0, 2],
                 [0, 0, 5, 0, 3, 7, 4, 0, 0],
                 [0, 0, 0, 7, 0, 0, 0, 2, 9],
                 [8, 2, 0, 0, 0, 0, 1, 7, 0],
                 [0, 5, 0, 9, 0, 6, 8, 0, 4]])
game = solve_sudoku(entree_facile)


[[9 1 6 4 7 5 2 8 7]
 [5 7 2 6 9 9 5 4 1]
 [3 4 8 6 7 2 6 9 6]
 [2 9 4 1 6 9 3 5 7]
 [7 3 1 5 4 8 9 6 2]
 [2 6 5 2 3 7 4 1 8]
 [4 6 3 7 1 4 6 2 9]
 [8 2 9 5 5 4 1 7 5]
 [1 5 7 9 2 6 8 3 4]]
[[9. 0. 6. 4. 0. 5. 0. 8. 0.]
 [0. 7. 2. 0. 0. 0. 0. 4. 1.]
 [3. 4. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 4. 1. 6. 0. 3. 0. 0.]
 [7. 0. 1. 0. 4. 0. 9. 0. 2.]
 [0. 0. 5. 0. 3. 7. 4. 0. 0.]
 [0. 0. 0. 7. 0. 0. 0. 2. 9.]
 [8. 2. 0. 0. 0. 0. 1. 7. 0.]
 [0. 5. 0. 9. 0. 6. 8. 0. 4.]]
[[False  True False False  True False  True False  True]
 [ True False False  True  True  True  True False False]
 [False False  True  True  True False  True  True  True]
 [ True  True False False False  True False  True  True]
 [False  True False  True False  True False  True False]
 [ True  True False  True False False False  True  True]
 [ True  True  True False  True  True  True False False]
 [False False  True  True  True  True False False  True]
 [ True False  True False  True False False  True False]]
[[1.   1.   1.   1.   0.7

In [8]:
print('solved puzzle:\n')
print(game)

solved puzzle:

[[9 1 6 4 7 5 2 8 7]
 [5 7 2 6 9 9 5 4 1]
 [3 4 8 6 7 2 6 9 6]
 [2 9 4 1 6 9 3 5 7]
 [7 3 1 5 4 8 9 6 2]
 [2 6 5 2 3 7 4 1 8]
 [4 6 3 7 1 4 6 2 9]
 [8 2 9 5 5 4 1 7 5]
 [1 5 7 9 2 6 8 3 4]]


In [15]:
print(game.sum(axis = 1))
print(game.sum(axis = 0))


[49 48 51 46 45 38 42 46 45]
[41 43 45 45 44 54 44 45 49]


In [11]:
#tordu = np.array([[0, 0, 0, 0, 0, 0, 3, 0, 0], [1, 0, 0, 4, 0, 0, 0, 6, 0], [7, 0, 0, 6, 1, 0, 0, 9, 0], [0, 7, 0, 0, 3, 0, 0, 0, 0], [0, 8, 0, 0, 4, 0, 2, 0, 0], [3, 0, 0, 7, 0, 9, 1, 8, 0], [0, 1, 8, 0, 0, 0, 0, 0, 2], [0, 2, 0, 5, 0, 0, 0, 4, 0], [0, 0, 3, 0, 0, 0, 0, 1, 0]])
tordu = np.array( [
[8,0,0,9,0,0, 3,2,1],
[1,0,0,4,0,0, 7,6,0],
[7,0,0,6,1,0, 0,9,0],
[0,7,1,0,3,0, 0,5,0],
[0,8,0,1,4,0, 2,3,7],
[3,0,0,7,0,9, 1,8,0],
[0,1,8,3,0,0, 0,7,2],
[0,2,7,5,0,1, 0,4,3],
[0,0,3,0,0,0, 0,1,0]
])
game = solve_sudoku(tordu)
print(game)


59
7 6 4
[[8. 0. 0. 9. 0. 0. 3. 2. 1.]
 [1. 0. 0. 4. 0. 0. 7. 6. 0.]
 [7. 0. 0. 6. 1. 0. 0. 9. 0.]
 [0. 7. 1. 0. 3. 0. 0. 5. 0.]
 [0. 8. 0. 1. 4. 0. 2. 3. 7.]
 [3. 0. 0. 7. 0. 9. 1. 8. 0.]
 [0. 1. 8. 3. 0. 4. 0. 7. 2.]
 [0. 2. 7. 5. 0. 1. 0. 4. 3.]
 [0. 0. 3. 0. 0. 0. 0. 1. 0.]]
[[8 6 6 9 7 7 3 2 1]
 [1 3 2 4 8 3 7 6 8]
 [7 3 2 6 1 3 4 9 4]
 [2 7 1 8 3 8 4 5 9]
 [5 8 9 1 4 5 2 3 7]
 [3 5 2 7 5 9 1 8 4]
 [4 1 8 3 9 4 9 7 2]
 [6 2 7 5 8 1 8 4 3]
 [4 4 3 2 2 2 5 1 8]]


In [6]:
59//9


6

In [7]:
59%9

5